In [ ]:
# 「リソース」クライアントを作成
import boto3
dynamodb = boto3.resource('dynamodb')

In [ ]:
# テーブルの削除
# files = dynamodb.Table('Files')
# files.delete()

In [ ]:
# 全項目の削除
# files = dynamodb.Table('Files')
# for item in files.scan()['Items']:
#     files.delete_item(Key={'Folder': item['Folder'], 'File': item['File']})

In [ ]:
# テーブルを作成
dynamodb.create_table(
    TableName='Files', # テーブル名
    # 主キーの設定。HASHは「パーティションキー」を、RANGEは「ソートキー」を表す。
    KeySchema=[{'AttributeName': 'Folder', 'KeyType': 'HASH'},
              {'AttributeName': 'File', 'KeyType': 'RANGE'}], 
    # 主キーのデータ型の指定。「S」は String を表す。
    AttributeDefinitions=[
        {'AttributeName': 'Folder', 'AttributeType': 'S'},
        {'AttributeName': 'File', 'AttributeType': 'S'},
        {'AttributeName': 'Size', 'AttributeType': 'N'},
        {'AttributeName': 'Type', 'AttributeType': 'S'}
    ], 
     # スループットの指定。
    ProvisionedThroughput={'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10},
    LocalSecondaryIndexes=[
        # 「Size」（サイズ）でソートするためのLSI
        {
            'IndexName': 'FilesOrderBySize',
            'KeySchema': [
                {'AttributeName': 'Folder', 'KeyType': 'HASH'},
                {'AttributeName': 'Size', 'KeyType': 'RANGE'},
            ],
            'Projection': {'ProjectionType': 'ALL'}
        },        
        # 「Type」でソートするためのLSI
        {
            'IndexName': 'FilesOrderByType',
            'KeySchema': [
                {'AttributeName': 'Folder', 'KeyType': 'HASH'},
                {'AttributeName': 'Type', 'KeyType': 'RANGE'},
            ],
            'Projection': {'ProjectionType': 'ALL'}
        },        
    ],
    GlobalSecondaryIndexes=[
        # 「Type」（ファイル種別）でクエリを行うためのGSI
        {
            'IndexName': 'FileTypes',
            'KeySchema': [
                {'AttributeName': 'Type', 'KeyType': 'HASH'},
                {'AttributeName': 'Size', 'KeyType': 'RANGE'},
            ],
            'Projection': {'ProjectionType': 'ALL'},
            'ProvisionedThroughput': {'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10},
        },        
    ]
)

In [ ]:
# テーブルの取得
files = dynamodb.Table('Files')

In [ ]:
# テーブルの状態を確認。
# 特にセカンダリインデックスを伴うテーブル作成には時間がかかる
# 作成中は'CREATING'、作成完了すると'ACTIVE'になる。
files.table_status

In [ ]:
# 作成されるまで待つ。
files.wait_until_exists()
print('ok')

In [ ]:
# バッチで複数の項目を書き込み。
# batch_writer()でバッチ用オブジェクトを作り、それに対してput_item()を（複数回）呼び出す。
with files.batch_writer() as batch:
    batch.put_item(Item={'Folder': '/home/taro/', 'File': 'hello.c', 'Size': 100, 'Type': 'text'})
    batch.put_item(Item={'Folder': '/home/taro/', 'File': 'cat1.jpg', 'Size': 200, 'Type': 'image'})
    batch.put_item(Item={'Folder': '/home/taro/', 'File': 'cat2.jpg', 'Size': 300, 'Type': 'image'})
    batch.put_item(Item={'Folder': '/home/taro/', 'File': 'dog.jpg', 'Size': 400, 'Type': 'image'})
    batch.put_item(Item={'Folder': '/home/jiro/', 'File': 'Foo.java', 'Size': 100, 'Type': 'text'})
    batch.put_item(Item={'Folder': '/home/jiro/', 'File': 'Bar.java', 'Size': 200, 'Type': 'text'})
    batch.put_item(Item={'Folder': '/home/jiro/', 'File': 'Baz.java', 'Size': 300, 'Type': 'text'})
    

In [ ]:
# 書き込まれた項目を取り出して確認。
# scanは、すべての項目を無条件で取り出す。RCUを大量消費するので注意。
files.scan()['Items']

In [ ]:
# フォルダ「/home/taro」のファイルをすべて取り出す
files.query(
    KeyConditionExpression='#Folder = :Folder', # キー条件式
    ExpressionAttributeNames={'#Folder': 'Folder'}, # 式の属性名
    ExpressionAttributeValues={':Folder': '/home/taro/'} # 式の属性値
)['Items']

In [ ]:
# フォルダ「/home/taro」の、
# ファイル名が「hello.c」であるものを検索
files.query(
    KeyConditionExpression='#Folder = :Folder and #File = :File', # キー条件式
    ExpressionAttributeNames={'#Folder': 'Folder', '#File': 'File'}, # 式の属性名
    ExpressionAttributeValues={':Folder': '/home/taro/', ':File': 'hello.c'} # 式の属性値
)['Items']

In [ ]:
# セカンダリインデックス(LSI)の利用例。
# フォルダ「/home/taro」のファイルを
# サイズの小さい順に表示。
files.query(
    IndexName='FilesOrderBySize', # 使用するインデックスの指定
    KeyConditionExpression='#Folder = :Folder', # キー条件式
    ExpressionAttributeNames={'#Folder': 'Folder'}, # 式の属性名
    ExpressionAttributeValues={':Folder': '/home/taro/'} # 式の属性値
)['Items']

In [ ]:
# セカンダリインデックス(LSI)の利用例。
# フォルダ「/home/taro」のファイルを
# サイズの大きい順に表示。
files.query(
    IndexName='FilesOrderBySize', # 使用するインデックスの指定
    ScanIndexForward=False, # 逆順の指定
    KeyConditionExpression='#Folder = :Folder', # キー条件式
    ExpressionAttributeNames={'#Folder': 'Folder'}, # 式の属性名
    ExpressionAttributeValues={':Folder': '/home/taro/'} # 式の属性値
)['Items']

In [ ]:
# セカンダリインデックス(LSI)の利用例。
# フォルダ「/home/taro」の、
# サイズが200以上300以下であるファイルを検索
files.query(
    IndexName='FilesOrderBySize', # 使用するインデックスの指定
    KeyConditionExpression='#Folder = :Folder AND (#Size BETWEEN :MinSize AND :MaxSize)', # キー条件式
    ExpressionAttributeNames={'#Folder': 'Folder', '#Size': 'Size'}, # 式の属性名
    ExpressionAttributeValues={':Folder': '/home/taro/', ':MinSize': 200, ':MaxSize': 300} # 式の属性値
)['Items']

In [ ]:
# セカンダリインデックス(LSI)の利用例。
# フォルダ「/home/taro」のファイルを、
# Type順に並び替えて出力
files.query(
    IndexName='FilesOrderByType', # 使用するインデックスの指定
    KeyConditionExpression='#Folder = :Folder', # キー条件式
    ExpressionAttributeNames={'#Folder': 'Folder'}, # 式の属性名
    ExpressionAttributeValues={':Folder': '/home/taro/'} # 式の属性値
)['Items']

In [ ]:
# セカンダリインデックス(LSI)の利用例。
# フォルダ「/home/taro」のファイルで、
# Typeが「image」であるものを検索
files.query(
    IndexName='FilesOrderByType', # 使用するインデックスの指定
    KeyConditionExpression='#Folder = :Folder AND #Type = :Type', # キー条件式
    ExpressionAttributeNames={'#Folder': 'Folder', '#Type': 'Type'}, # 式の属性名
    ExpressionAttributeValues={':Folder': '/home/taro/', ':Type': 'image'} # 式の属性値
)['Items']

In [ ]:
# セカンダリインデックス(GSI)の利用例。
# 全項目から、Typeが「image」であるものを検索
files.query(
    IndexName='FileTypes', # 使用するインデックスの指定
    KeyConditionExpression='#Type = :Type', # キー条件式
    ExpressionAttributeNames={'#Type': 'Type'}, # 式の属性名
    ExpressionAttributeValues={':Type': 'text'} # 式の属性値
)['Items']